In [1]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath('..'))  # Go up one directory if notebook is in /notebooks
from src.preprocessing import create_dataloader
from src.config import TrainingConfig
from src.eval import Eval
from diffusers import DiTPipeline
from src.vae import DummyAutoencoderKL
from src.DiT import create_model, create_noise_scheduler
import torch

/vol/bitbucket/aw624/Low_Rank_Generative_Models/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataloader = create_dataloader("uoft-cs/cifar10", "train", TrainingConfig())

Loaded dataset uoft-cs/cifar10 train split with 50000 images


In [3]:
for batch in train_dataloader:
    print(batch)
    break

{'img': tensor([[[[-0.3490, -0.3490, -0.3647,  ...,  0.2471,  0.2157,  0.2157],
          [-0.3098, -0.3333, -0.3490,  ...,  0.2549,  0.0824,  0.1373],
          [-0.3020, -0.3176, -0.3176,  ...,  0.1843,  0.0353,  0.1922],
          ...,
          [-0.3490, -0.2549, -0.2235,  ..., -0.2392, -0.2078, -0.2627],
          [-0.2784, -0.2471, -0.2627,  ..., -0.2706, -0.2471, -0.2627],
          [-0.2471, -0.3176, -0.2941,  ..., -0.2471, -0.2549, -0.2627]],

         [[-0.1765, -0.1765, -0.1922,  ...,  0.3412,  0.2784,  0.2549],
          [-0.1373, -0.1608, -0.1765,  ...,  0.3569,  0.1686,  0.1922],
          [-0.1294, -0.1451, -0.1451,  ...,  0.3176,  0.1373,  0.2706],
          ...,
          [-0.1686, -0.0745, -0.0431,  ..., -0.0588, -0.0275, -0.0824],
          [-0.0980, -0.0667, -0.0824,  ..., -0.0902, -0.0667, -0.0824],
          [-0.0667, -0.1373, -0.1137,  ..., -0.0667, -0.0745, -0.0824]],

         [[-0.3412, -0.3412, -0.3647,  ...,  0.2863,  0.2392,  0.2157],
          [-0.3020, -0

In [2]:
model = create_model(TrainingConfig())
print(TrainingConfig().image_size)
model.load_state_dict(torch.load(Path("/vol/bitbucket/aw624/Low_Rank_Generative_Models/logs/DiT20250423_205135/model.pt")))
model.to("cuda")
model.eval()

pipeline = DiTPipeline(
    transformer=model,
    scheduler=create_noise_scheduler(TrainingConfig()),
    vae=DummyAutoencoderKL(),
)

32


In [ ]:
eval = Eval

In [3]:
batch_num = 10000//64

generated_images = []

for i in range(1):

    image = pipeline(
        class_labels=torch.zeros(64, dtype=torch.long),
                num_inference_steps=1000,
                output_type="tensor"
            ).images
    
    generated_images.extend(image)



/vol/bitbucket/aw624/Low_Rank_Generative_Models/venv/lib/python3.12/site-packages/diffusers/pipelines/dit/pipeline_dit.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_labels = torch.tensor(class_labels, device=self._execution_device).reshape(-1)
100%|██████████| 1000/1000 [00:36<00:00, 27.04it/s]


In [4]:
generated_images = torch.tensor(generated_images)

print(generated_images.shape)

generated_images = generated_images.permute(0, 3, 1, 2)


/tmp/ipykernel_1068804/1114798230.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  generated_images = torch.tensor(generated_images)


torch.Size([64, 32, 32, 3])


In [5]:
help(DiTPipeline.from_pretrained)

Help on method from_pretrained in module diffusers.pipelines.pipeline_utils:

from_pretrained(pretrained_model_name_or_path: Union[str, os.PathLike, NoneType], **kwargs) -> Self class method of diffusers.pipelines.dit.pipeline_dit.DiTPipeline
    Instantiate a PyTorch diffusion pipeline from pretrained pipeline weights.

    The pipeline is set in evaluation mode (`model.eval()`) by default.

    If you get the error message below, you need to finetune the weights for your downstream task:

    ```
    Some weights of UNet2DConditionModel were not initialized from the model checkpoint at stable-diffusion-v1-5/stable-diffusion-v1-5 and are newly initialized because the shapes did not match:
    - conv_in.weight: found shape torch.Size([320, 4, 3, 3]) in the checkpoint and torch.Size([320, 9, 3, 3]) in the model instantiated
    You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
    ```

    Parameters:
        pretrained_model_

In [3]:
test_dataloader = create_dataloader("uoft-cs/cifar10", "test", TrainingConfig())

Loaded dataset uoft-cs/cifar10 test split with 10000 images


In [7]:
for batch in test_dataloader:
    print((batch["img"].dtype))
    print(batch["img"].shape)
    break

torch.float32
torch.Size([64, 3, 32, 32])


In [4]:
eval = Eval(test_dataloader, eval_dataset_size=100)

Computing real features: 100%|██████████| 2/2 [00:01<00:00,  1.47it/s]


In [5]:
eval.compute_metrics(pipeline)

/vol/bitbucket/aw624/Low_Rank_Generative_Models/venv/lib/python3.12/site-packages/diffusers/pipelines/dit/pipeline_dit.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_labels = torch.tensor(class_labels, device=self._execution_device).reshape(-1)
  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:37<00:00, 27.00it/s]


{'fid': 64.75636291503906}

In [10]:
eval.compute_metrics(generated_images)

{'fid': 60.301753997802734}

In [9]:
type(test_dataloader.dataset.data[0])

pyarrow.lib.ChunkedArray